## Modelowanie układów przepływowych - wykład 
#### (budowa solwera równań płytkiej wody na bazie pakietu PyMPDATA)
Sylwester Arabas (sylwester.arabas@agh.edu.pl)

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA')

### 0. potrzebne pakiety Pythona

In [ ]:
import numpy as np
from matplotlib import pyplot
from open_atmos_jupyter_utils import show_anim
from PyMPDATA import ScalarField, Solver, Stepper, VectorField, Options
from PyMPDATA.boundary_conditions import Constant

### 1. opis układu: symbole i równania

$$ \zeta(t; x,y) \rightarrow \text{wysokość swobodnej powierzchni względem geoidy (z=0)}$$
$$ b(x,y) \rightarrow \text{batymetria mierzona dodatnio w doół od geoidy}$$
$$ h(t; x,y) = \zeta + b \rightarrow \text{całkowita głębokość kolumny wody} $$
$$ \vec{u} = [u, v]$$
$$
\begin{cases}
  \partial_t h &=&\! -\nabla \cdot (\vec{u}h)\\
  \partial_t (hu) &=&\! -\nabla \cdot (\vec{u}hu) - gh\partial_x\zeta \\
  \partial_t (hv) &=&\! -\nabla \cdot (\vec{u}hv) - gh\partial_y\zeta
\end{cases}
$$

### 2. solwer "hello-world" zbudowany na bazie PyMPDATA (b=0)

In [ ]:
class ShallowWaterEquationsIntegrator:
    def __init__(self, *, h_initial: np.ndarray, options: Options):
        X, Y, grid = 0, 1, h_initial.shape
        kwargs = { 
            "boundary_conditions": [Constant(value=0)] * len(grid),
            "halo": options.n_halo,
        }
        self.advector = VectorField((
            np.zeros((grid[X] + 1, grid[Y])),
            np.zeros((grid[X], grid[Y] + 1)) 
        ), **kwargs)
        stepper = Stepper(options=options, grid=grid)
        self.solvers = { k: Solver(stepper, v, self.advector) for k, v in {
            "h": ScalarField(h_initial, **kwargs),
            "uh": ScalarField(np.zeros(grid), **kwargs),
            "vh": ScalarField(np.zeros(grid), **kwargs),
        }.items() }

    def __getitem__(self, key):
        return self.solvers[key].advectee.get()

    def _apply_half_rhs(self, *, key, axis, g_times_dt_over_dxy):
        self[key][:] -= .5 * g_times_dt_over_dxy * self['h'] * np.gradient(self['h'], axis=axis)

    def _update_courant_numbers(self, *, axis, key, mask, dt_over_dxy):
        velocity = np.where(mask, np.nan, 0)
        momentum = self[key]
        np.divide(momentum, self['h'], where=mask, out=velocity)

        whole = slice(None, None) 
        all_but_last = slice(None, -1)
        all_but_first_and_last = slice(1, -1)

        velocity_at_cell_boundaries = velocity[( 
            (all_but_last, whole),
            (whole, all_but_last),
        )[axis]] + np.diff(velocity, axis=axis) / 2 
        courant_number = self.advector.get_component(axis)[(
            (all_but_first_and_last, whole),
            (whole, all_but_first_and_last)
        )[axis]]
        courant_number[:] = velocity_at_cell_boundaries * dt_over_dxy[axis]
        assert np.amax(np.abs(courant_number)) <= 1

    def __call__(self, *, nt: int, g: float, dt_over_dxy: tuple, outfreq: int, eps: float=1e-7):
        output = {k: [] for k in self.solvers.keys()}
        for it in range(nt + 1): 
            if it != 0:
                mask = self['h'] > eps
                for axis, key in enumerate(("uh", "vh")):
                    self._update_courant_numbers(axis=axis, key=key, mask=mask, dt_over_dxy=dt_over_dxy)
                self.solvers["h"].advance(n_steps=1)
                for axis, key in enumerate(("uh", "vh")):
                    self._apply_half_rhs(key=key, axis=axis, g_times_dt_over_dxy=g * dt_over_dxy[axis])
                    self.solvers[key].advance(n_steps=1)
                    self._apply_half_rhs(key=key, axis=axis, g_times_dt_over_dxy=g * dt_over_dxy[axis])
            if it % outfreq == 0:
                for key in self.solvers.keys():
                    output[key].append(self[key].copy())
        return output

### 3. Przykładowa symulacja

In [ ]:
h_initial = .5 * np.ones(grid := (50, 40))
h_initial[
    grid[0] // 2 - grid[0] // 20:
    grid[0] // 2 + grid[0] // 20,
    grid[1] // 2 - grid[1] // 20:
    grid[1] // 2 + grid[1] // 20
] += .025
output = ShallowWaterEquationsIntegrator(
    h_initial=h_initial,
    options=Options(nonoscillatory=True, infinite_gauge=True),
)(
    nt=30, g=10, dt_over_dxy=(.25, .25), outfreq=(outfreq := 3)
)

### 4. Wizualizacja

In [ ]:
def plot(frame, *, zlim=(.45, .55)):
    psi = output['h'][frame]
    xi, yi = np.indices(psi.shape)
    fig, ax = pyplot.subplots(subplot_kw={"projection": "3d"}, figsize=(12, 6))
    ax.plot_wireframe(xi+.5, yi+.5, psi, color='blue', linewidth=.5)
    ax.set(zlim=zlim, proj_type='ortho', title=f"t / Δt = {frame * outfreq}", zlabel="$\zeta$")
    for axis in (ax.xaxis, ax.yaxis, ax.zaxis):
        axis.pane.fill = False
        axis.pane.set_edgecolor('black')
        axis.pane.set_alpha(1)
    for axis in ('x', 'y'):
        getattr(ax, f'set_{axis}label')(f"{axis} / Δ{axis}")
    return fig
show_anim(plot, range(len(output['h'])))